In [ ]:
import pandas as pd

In [ ]:
df_tweets = pd.read_csv('/Users/yassinkortam/Documents/GitHub/segmentation-pipeline/data/tweets/tokyo_2020_tweets.csv', low_memory=False).sample(n=10000, replace=False, random_state=42)
df_athletes = pd.read_excel('/Users/yassinkortam/Documents/GitHub/segmentation-pipeline/data/olympics/Athletes.xlsx')
df_medals = pd.read_excel('/Users/yassinkortam/Documents/GitHub/segmentation-pipeline/data/olympics/Medals.xlsx')

In [ ]:
df_medals.sort_values(by='Rank', ascending=True)

In [ ]:
df_athletes.head(3)

In [ ]:
df_merged = df_medals.merge(df_athletes, left_on='Team/NOC', right_on='NOC').drop(columns=df_medals.columns)
df_merged

In [ ]:
df_tweets.head(3)